# Pandas DataFrame operations

This is a reference notebook for useful operations on python pandas package's dataframe data type. 



In [2]:
import pandas as pd

## Initialization / population

In [3]:
df = pd.DataFrame()

In [4]:
df = pd.DataFrame({"A": range(3)})
df

,A
0,0
1,1
2,2


In [5]:
# Empty dataframe with heading
df = pd.DataFrame(columns=['A','B','C', 'D'])
df

,A,B,C,D


In [6]:
values1 = ['VW', 'Toyota', 'Tesla', 'VW']
values2 = ['Beetle', 'Corolla', 'Model S', 'Rabbit']
values3 = [1972, 2005, 2016, 2009]
df = pd.DataFrame({'Company':values1, 'Model':values2, 'Year':values3})
df

,Company,Model,Year
0,VW,Beetle,1972
1,Toyota,Corolla,2005
2,Tesla,Model S,2016
3,VW,Rabbit,2009


In [7]:
# Access one cell
df.loc[1,"Model"]

'Corolla'

## Add data

In [8]:
new_df = pd.DataFrame({'Company':['Jeep', 'Chevrolet'], 'Model':['Cherokee', 'Impala'], 'Year':[2007, 2004]})
df = df.append(new_df)   ### Need to assign back to df, otherwise the changes are lost
df

,Company,Model,Year
0,VW,Beetle,1972
1,Toyota,Corolla,2005
2,Tesla,Model S,2016
3,VW,Rabbit,2009
0,Jeep,Cherokee,2007
1,Chevrolet,Impala,2004


In [9]:
df = df.append({'Company':'Nissan', 'Model':'Leaf', 'Year':2017}, ignore_index=True)
df

,Company,Model,Year
0,VW,Beetle,1972
1,Toyota,Corolla,2005
2,Tesla,Model S,2016
3,VW,Rabbit,2009
4,Jeep,Cherokee,2007
5,Chevrolet,Impala,2004
6,Nissan,Leaf,2017


#### Modify a value in the df

In [10]:
df.at[3,'Year'] = 2010
df

,Company,Model,Year
0,VW,Beetle,1972
1,Toyota,Corolla,2005
2,Tesla,Model S,2016
3,VW,Rabbit,2010
4,Jeep,Cherokee,2007
5,Chevrolet,Impala,2004
6,Nissan,Leaf,2017


#### Add column

In [11]:
df['Year_Squared'] = df['Year']**2
df

,Company,Model,Year,Year_Squared
0,VW,Beetle,1972,3888784
1,Toyota,Corolla,2005,4020025
2,Tesla,Model S,2016,4064256
3,VW,Rabbit,2010,4040100
4,Jeep,Cherokee,2007,4028049
5,Chevrolet,Impala,2004,4016016
6,Nissan,Leaf,2017,4068289


#### Delete column

In [12]:
del df['Year_Squared']
df

,Company,Model,Year
0,VW,Beetle,1972
1,Toyota,Corolla,2005
2,Tesla,Model S,2016
3,VW,Rabbit,2010
4,Jeep,Cherokee,2007
5,Chevrolet,Impala,2004
6,Nissan,Leaf,2017


##### Delete several columns by name

In [4]:
del_df = pd.DataFrame({"col1":range(3), "col2":range(3), "col3":range(3)})
del_df

,col1,col2,col3
0,0,0,0
1,1,1,1
2,2,2,2


In [6]:
del_df = del_df.drop(["col1", "col2"], axis=1)
del_df

,col3
0,0
1,1
2,2


#### Add row

In [13]:
df.loc[3] = ['Subaru','Forrester', 2010]  # Adds this row INSTEAD OF 4th row in data frame
df

,Company,Model,Year
0,VW,Beetle,1972
1,Toyota,Corolla,2005
2,Tesla,Model S,2016
3,Subaru,Forrester,2010
4,Jeep,Cherokee,2007
5,Chevrolet,Impala,2004
6,Nissan,Leaf,2017


#### Delete row

In [14]:
df.drop(df.index[[2]])  # drops the row indexed as 2. 
                        # Note: once it is dropped, the index 2 is gone. The rest of the indexes are not shifted

,Company,Model,Year
0,VW,Beetle,1972
1,Toyota,Corolla,2005
3,Subaru,Forrester,2010
4,Jeep,Cherokee,2007
5,Chevrolet,Impala,2004
6,Nissan,Leaf,2017


### Add column and add values to this column with an index

Use case: we want to add a "rating" column and add rating for Toyota (5) and Jeep (3). Note that because we have deleted a row, the index is missing value 2.

In [15]:
from pandas import *
idx = pd.Int64Index([1, 4])
tmp_df = DataFrame(index = idx, data =({'rating':[5,3]}))
rating_df = df.join(tmp_df)
rating_df

,Company,Model,Year,rating
0,VW,Beetle,1972,NaN
1,Toyota,Corolla,2005,5.0
2,Tesla,Model S,2016,NaN
3,Subaru,Forrester,2010,NaN
4,Jeep,Cherokee,2007,3.0
5,Chevrolet,Impala,2004,NaN
6,Nissan,Leaf,2017,NaN


### Drop rows with empty values

##### Add another col with empy data for an example

In [16]:
tmp_df = DataFrame(index = pd.Int64Index([0, 4]), data =({'some_col':[0,0]}))
tmp_df = rating_df.join(tmp_df)
tmp_df



,Company,Model,Year,rating,some_col
0,VW,Beetle,1972,NaN,0.0
1,Toyota,Corolla,2005,5.0,NaN
2,Tesla,Model S,2016,NaN,NaN
3,Subaru,Forrester,2010,NaN,NaN
4,Jeep,Cherokee,2007,3.0,0.0
5,Chevrolet,Impala,2004,NaN,NaN
6,Nissan,Leaf,2017,NaN,NaN


##### Drop (note that everything but Jeep is dropped, because dropna works on all columns)

In [17]:
tmp_df.dropna()

,Company,Model,Year,rating,some_col
4,Jeep,Cherokee,2007,3.0,0.0


#### To "drop" only based on one column, use select non-empty instead

In [18]:
tmp_df[pd.notnull(tmp_df['rating'])]

,Company,Model,Year,rating,some_col
1,Toyota,Corolla,2005,5.0,NaN
4,Jeep,Cherokee,2007,3.0,0.0


### Apply complex condition to select a subset of data frame

Scenario: want to drop the rows where two columns have the same value.
For this I need to use an .apply(). apply is used to send a column to a function. To send a row to a function need to add axis=1 to apply parameters.

In [19]:
# Want to remove second row
values1 = ['Something', 'Something1', 'DUDU', 'VW']
values2 = ['Other', 'Something1', 'Lala', 'Rabbit']
dupl_df = pd.DataFrame({'col1':values1, 'col2':values2})
dupl_df

,col1,col2
0,Something,Other
1,Something1,Something1
2,DUDU,Lala
3,VW,Rabbit


In [20]:
# Don't forget to assign this to a data frame, if you want to keep this 
dupl_df[dupl_df.apply(lambda row: row["col1"]!=row["col2"], axis=1)]

,col1,col2
0,Something,Other
2,DUDU,Lala
3,VW,Rabbit


In [21]:
dupl_df

,col1,col2
0,Something,Other
1,Something1,Something1
2,DUDU,Lala
3,VW,Rabbit


### Iterate over a column

In [22]:
for md in df['Model']:
    print(md)

Beetle
Corolla
Model S
Forrester
Cherokee
Impala
Leaf


In [23]:
## Doesn't work
'''
for md in df[0]:
    print(md)
'''

'\nfor md in df[0]:\n    print(md)\n'

In [24]:
### Iterate over rows in a data frame
for index, row in df.iterrows():
    print('Index: {}\nRow: {}\n'.format(index,row))

Index: 0
Row: Company        VW
Model      Beetle
Year         1972
Name: 0, dtype: object

Index: 1
Row: Company     Toyota
Model      Corolla
Year          2005
Name: 1, dtype: object

Index: 2
Row: Company      Tesla
Model      Model S
Year          2016
Name: 2, dtype: object

Index: 3
Row: Company       Subaru
Model      Forrester
Year            2010
Name: 3, dtype: object

Index: 4
Row: Company        Jeep
Model      Cherokee
Year           2007
Name: 4, dtype: object

Index: 5
Row: Company    Chevrolet
Model         Impala
Year            2004
Name: 5, dtype: object

Index: 6
Row: Company    Nissan
Model        Leaf
Year         2017
Name: 6, dtype: object



## Exploring /viewing

In [25]:
# Load decent data to view
df = pd.read_excel('data/test.xlsx', sheet_name='Sheet1')

In [26]:
pd.set_option('display.max_rows', 150)
df

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,Abortiporus biennis,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,Absidia anomala,Hesseltine & J.J. Ellis,MycoBank
3,Apophysomyces atrospora,Absidia blakesleeana var. atrospora,H. Naganishi & Hirahara,MycoBank
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank
5,Absidia californica,Absidia californica,J.J. Ellis & Hesseltine,MycoBank
6,Absidia coerulea,Absidia coerulea,Bainier,MycoBank
7,Absidia corymbifera,Lichtheimia corymbifera,(Cohn) Saccardo & Trotter,MycoBank
8,Absidia cuneospora,Absidia cuneospora,G.F. Orr & Plunkett,MycoBank
9,Absidia cylindrospora var. cylindrospora,Absidia cylindrospora var. cylindrospora,NaN,MycoBank


In [27]:
df.columns

Index(['taxon_name', 'current_taxon_name', 'authors', 'source'], dtype='object')

In [28]:
df.dtypes

taxon_name            object
current_taxon_name    object
authors               object
source                object
dtype: object

In [29]:
df.shape

(205, 4)

In [30]:
df.describe()

,taxon_name,current_taxon_name,authors,source
count,205,205,203,205
unique,196,76,167,3
top,Alternaria brassicae,Plenodomus lingam,Frisch & G. Thor,ICTF
freq,4,16,6,157


In [31]:
df.describe().transpose()

,count,unique,top,freq
taxon_name,205,196,Alternaria brassicae,4
current_taxon_name,205,76,Plenodomus lingam,16
authors,203,167,Frisch & G. Thor,6
source,205,3,ICTF,157


In [32]:
df.head()   #excel_df.head(10)

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,Abortiporus biennis,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,Absidia anomala,Hesseltine & J.J. Ellis,MycoBank
3,Apophysomyces atrospora,Absidia blakesleeana var. atrospora,H. Naganishi & Hirahara,MycoBank
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
taxon_name            205 non-null object
current_taxon_name    205 non-null object
authors               203 non-null object
source                205 non-null object
dtypes: object(4)
memory usage: 6.5+ KB


#### Slicing and dicing

In [34]:
df[:3] # first rows

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,Abortiporus biennis,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,Absidia anomala,Hesseltine & J.J. Ellis,MycoBank


In [35]:
df[-4:] # last rows

,taxon_name,current_taxon_name,authors,source
201,Helminthosporium acrothecioides,Bipolaris sorokiniana,Lindf.,ICTF
202,Helminthosporium californicum,Bipolaris sorokiniana,Mackie & G.E. Paxton,ICTF
203,Ophiobolus sativus,Bipolaris sorokiniana,S. Ito & Kurib.,ICTF
204,Cochliobolus sativus,Bipolaris sorokiniana,(S. Ito & Kurib.) Drechsler ex Dastur,ICTF


In [36]:
df[3:5]  #from to

,taxon_name,current_taxon_name,authors,source
3,Apophysomyces atrospora,Absidia blakesleeana var. atrospora,H. Naganishi & Hirahara,MycoBank
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank


### Subset based on cell value
Usecase: want to select all the rows that have 'ICTF' as a value in the source column

In [41]:
df_sub = df.loc[df['source']=='ICTF']
#df_sub.head()
len(df_sub)

157

In [42]:
# Or if we are looking for a set of values
values = ['ICTF', 'MycoBank']
df_sub = df.loc[df['source'].isin(values)]
len(df_sub)
#df_sub.head()

184

#### Group by

In [42]:
#pl.groupby('some_val').mean()

## Working with

Note that adding one row at a time to df is computationally expensive, so better to use other datastructures to collect data and add it all in one go.

In [43]:
df[:5]

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,Abortiporus biennis,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,Absidia anomala,Hesseltine & J.J. Ellis,MycoBank
3,Apophysomyces atrospora,Absidia blakesleeana var. atrospora,H. Naganishi & Hirahara,MycoBank
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank


### Create a new data frame with a subset of two columns from the original

In [47]:
species_disease_df = df[['taxon_name','source']]
species_disease_df.head()

,taxon_name,source
0,Abortiporus biennis,MycoBank
1,Polyporus biennis,MycoBank
2,Absidia anomala,MycoBank
3,Apophysomyces atrospora,MycoBank
4,Absidia blakesleeana,MycoBank


#### Create a new series from one column of a dataframe

In [50]:
species_sr = df['taxon_name']
species_sr.head()   # .head() also works in series

0        Abortiporus biennis
1          Polyporus biennis
2            Absidia anomala
3    Apophysomyces atrospora
4       Absidia blakesleeana
Name: taxon_name, dtype: object

In [51]:
arr = []
arr.append([])
arr[0].append('aa1')
arr[0].append('aa2')
arr

[['aa1', 'aa2']]

In [52]:
import numpy as np

In [54]:
#df = np.DataFrame(arr)

#### Rename columns

In [2]:
df1 = pd.DataFrame({'old_1':[1,2], 'old_2':[3,4]})
df1

,old_1,old_2
0,1,3
1,2,4


In [4]:
df1.columns = ['new_1', 'new_2']
df1

,new_1,new_2
0,1,3
1,2,4


## Duplicates

### Find and explore

In [60]:
df = pd.read_excel("data/duplicates.xlsx")
df.head(10)

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,-,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,-,Hesseltine & J.J. Ellis,MycoBank
3,Absidia anomala,NaN,H. Naganishi & Hirahara,IndexFungorum
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank
5,Absidia californica,Absidia californica,J.J. Ellis & Hesseltine,MycoBank
6,Abortiporus biennis,Absidia coerulea,Bainier,IndexFungorum
7,Absidia corymbifera,Lichtheimia corymbifera,(Cohn) Saccardo & Trotter,MycoBank
8,Absidia cuneospora,Absidia cuneospora,G.F. Orr & Plunkett,MycoBank
9,Absidia cylindrospora var. cylindrospora,Absidia cylindrospora var. cylindrospora,NaN,MycoBank


In [62]:
df_dupl = df.duplicated('taxon_name', keep=False)
df_dupl.head()

0     True
1    False
2     True
3     True
4    False
dtype: bool

In [63]:
df.insert(loc=4, column='taxon_name duplicated', value=df_dupl)
df.head()

,taxon_name,current_taxon_name,authors,source,taxon_name duplicated
0,Abortiporus biennis,-,(Bull.) Singer,MycoBank,True
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank,False
2,Absidia anomala,-,Hesseltine & J.J. Ellis,MycoBank,True
3,Absidia anomala,NaN,H. Naganishi & Hirahara,IndexFungorum,True
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank,False


In [64]:
unique_names = df['taxon_name'].unique()
unique_names.sort()
unique_names

array(['Abortiporus biennis', 'Absidia anomala', 'Absidia blakesleeana',
       'Absidia californica', 'Absidia corymbifera', 'Absidia cuneospora',
       'Absidia cylindrospora var. cylindrospora',
       'Absidia cylindrospora var. nigra',
       'Absidia cylindrospora var. rhizomorpha', 'Absidia glauca',
       'Absidia gracilis', 'Absidia griseola', 'Absidia hesseltinei',
       'Polyporus biennis'], dtype=object)

In [65]:
for name in unique_names:
    group = df.loc[df['taxon_name'] == name]
    new_group = group.drop_duplicates('current_taxon_name')
    print(new_group)
    print('--------------------')

             taxon_name   current_taxon_name         authors         source  \
0   Abortiporus biennis                    -  (Bull.) Singer       MycoBank   
6   Abortiporus biennis     Absidia coerulea         Bainier  IndexFungorum   
17  Abortiporus biennis  Absidia heterospora         Y. Ling           ICTF   

    taxon_name duplicated  
0                    True  
6                    True  
17                   True  
--------------------
        taxon_name current_taxon_name                  authors         source  \
2  Absidia anomala                  -  Hesseltine & J.J. Ellis       MycoBank   
3  Absidia anomala                NaN  H. Naganishi & Hirahara  IndexFungorum   

   taxon_name duplicated  
2                   True  
3                   True  
--------------------
             taxon_name      current_taxon_name  authors    source  \
4  Absidia blakesleeana  Lichtheimia hyalospora  Lendner  MycoBank   

   taxon_name duplicated  
4                  False  
---------

In [66]:
# Convert column format
#pd.to_numeric(agr_land_area['Value'])

### Remove duplicates

In [67]:
df = pd.read_excel("data/duplicates.xlsx")
df

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,-,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,-,Hesseltine & J.J. Ellis,MycoBank
3,Absidia anomala,NaN,H. Naganishi & Hirahara,IndexFungorum
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank
5,Absidia californica,Absidia californica,J.J. Ellis & Hesseltine,MycoBank
6,Abortiporus biennis,Absidia coerulea,Bainier,IndexFungorum
7,Absidia corymbifera,Lichtheimia corymbifera,(Cohn) Saccardo & Trotter,MycoBank
8,Absidia cuneospora,Absidia cuneospora,G.F. Orr & Plunkett,MycoBank
9,Absidia cylindrospora var. cylindrospora,Absidia cylindrospora var. cylindrospora,NaN,MycoBank


In [68]:
df.drop_duplicates(subset='taxon_name', keep='last')

,taxon_name,current_taxon_name,authors,source
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
3,Absidia anomala,NaN,H. Naganishi & Hirahara,IndexFungorum
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank
7,Absidia corymbifera,Lichtheimia corymbifera,(Cohn) Saccardo & Trotter,MycoBank
8,Absidia cuneospora,Absidia cuneospora,G.F. Orr & Plunkett,MycoBank
9,Absidia cylindrospora var. cylindrospora,Absidia cylindrospora var. cylindrospora,NaN,MycoBank
10,Absidia cylindrospora var. nigra,Absidia cylindrospora var. nigra,Hesseltine & J.J. Ellis,MycoBank
11,Absidia cylindrospora var. rhizomorpha,Absidia cylindrospora var. rhizomorpha,Hesseltine & J.J. Ellis,MycoBank
12,Absidia californica,Absidia californica,Linnemann,ICTF
13,Absidia glauca,Absidia glauca,Hagem,MycoBank


## Loading data from Excel

#### Read Excel, way 1
This is a little faster, than way2

In [69]:
excel_obj = pd.ExcelFile("data/test.xlsx")
excel_df = excel_obj.parse('Sheet1')  # sheet name
excel_df.head()

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,Abortiporus biennis,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,Absidia anomala,Hesseltine & J.J. Ellis,MycoBank
3,Apophysomyces atrospora,Absidia blakesleeana var. atrospora,H. Naganishi & Hirahara,MycoBank
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank


#### Read Excel, way 2:
A little slower than way1

In [70]:
df = pd.read_excel('data/test.xlsx', sheet_name='Sheet1')
df.head()

,taxon_name,current_taxon_name,authors,source
0,Abortiporus biennis,Abortiporus biennis,(Bull.) Singer,MycoBank
1,Polyporus biennis,Abortiporus biennis,(Bulliard) Fries,MycoBank
2,Absidia anomala,Absidia anomala,Hesseltine & J.J. Ellis,MycoBank
3,Apophysomyces atrospora,Absidia blakesleeana var. atrospora,H. Naganishi & Hirahara,MycoBank
4,Absidia blakesleeana,Lichtheimia hyalospora,Lendner,MycoBank


## Load data from json query

#### Read json from URL, way 1
This is brittle, since it's using local solr url as an example. 
TODO: find an accessible json qry

In [72]:
query_url = 'http://localhost:8983/solr/CFIA_all/select?fl=id&q=title:grain'

In [78]:
'''
df2 = pd.read_json(query_url)
df2.head()
'''

'\ndf2 = pd.read_json(query_url)\ndf2.head()\n'

#### Read json from url, way 1

In [76]:
import requests
'''
query_url = 'http://localhost:8983/solr/CFIA_all/select?fl=id&q=title:grain'
r = requests.get(query_url)
query_response_df = pd.DataFrame(r.json()['response']['docs'])
query_response_df.head()
'''

"\nquery_url = 'http://localhost:8983/solr/CFIA_all/select?fl=id&q=title:grain'\nr = requests.get(query_url)\nquery_response_df = pd.DataFrame(r.json()['response']['docs'])\nquery_response_df.head()\n"

#### Read json from url, way 2 (no requests)
Note that this method does not use request or any other library, but pandas. The downside is that it reads into dataframe exactly what the url returned, so headers and everything. If you need to pre-process the json before, then the requests way above is a better option.

In [77]:
'''
df2 = pd.read_json(query_url)
df2
'''

'\ndf2 = pd.read_json(query_url)\ndf2\n'

## Write data to an excel file

In [ ]:
writer = pd.ExcelWriter('/path/to/file/dataframe.xlsx')
df.to_excel(excel_writer=writer, sheet_name='test_sheet')
writer.save()